In [1]:
from imp_act import make

## Environment

The illustration below introduces the abstractions and terminology used in the environment.

<img src="./environment_overview.png" alt="Description" style="width:80%;">

In [2]:
# initialize the environment
env = make("ToyExample-v2") # this environment has a different topology than the one above

# reset the environment
obs = env.reset()

### Accessing environment variables

In [3]:
## Graph

# We use the igraph library to represent the graph, which is stored in the env object
# For more information on igraph, see: https://python.igraph.org/en/stable/tutorial.html
g = env.graph

# We demonstrate how to access some basic properties of the graph
print(f"Number of nodes: {env.graph.vcount()}")
print(f"Number of edges: {env.graph.ecount()}")
print(f"Adjacency matrix: {g.get_adjacency().data}")
print()

## Road Network

# Each graph edge represents a unique road edge in the environment,
# and each road edge has multiple road segments.

for edge in env.graph.es:
    edge_index = edge.index

    # Each graph edge has a unique RoadEdge object associated with it
    road_edge = edge["road_edge"]

    # Each RoadEdge object has a list of RoadSegment objects
    num_segments = len(road_edge.segments)

    print(f"Edge {edge_index} has {num_segments} segments")

print()

# A road segment is the fundamental deteriorating unit of the road network.
# It has a damage state, observation, belief, capacity, and base travel time.

# Let us access the first road segment of the second road edge
road_edge = env.graph.es[1]["road_edge"]
road_segment = road_edge.segments[0]

print(f"Road segment 0 of edge 1 has the following properties:")
print(f"Damage state: {road_segment.state}")
print(f"Observation: {road_segment.observation}")
print(f"Belief: {road_segment.belief}")
print(f"Capacity: {road_segment.capacity}")
print(f"Base travel time: {road_segment.base_travel_time}")

Number of nodes: 6
Number of edges: 12
Adjacency matrix: [[0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1], [0, 1, 0, 0, 1, 0]]

Edge 0 has 1 segments
Edge 1 has 1 segments
Edge 2 has 1 segments
Edge 3 has 1 segments
Edge 4 has 1 segments
Edge 5 has 1 segments
Edge 6 has 1 segments
Edge 7 has 1 segments
Edge 8 has 1 segments
Edge 9 has 1 segments
Edge 10 has 1 segments
Edge 11 has 1 segments

Road segment 0 of edge 1 has the following properties:
Damage state: 0
Observation: 0
Belief: [0.85 0.05 0.05 0.05 0.  ]
Capacity: 50
Base travel time: 1


### Count redundancies in the network

This is a high-level summary of the redundancy count in the network. 

Internally, a function `get_count_redundancies_summary` takes as input, a graph and trips data specified for that environment. It counts all shortest paths for all trips in the network and subtracts 1 from the count to get the redundancy count.

In [4]:
print(env.get_count_redundancies_summary())

Summary | Network Trips

Total number of trips: 5

O: 0, D: 5 | # paths: 3
O: 5, D: 0 | # paths: 2
O: 1, D: 3 | # paths: 2
O: 4, D: 2 | # paths: 4
O: 2, D: 5 | # paths: 1

Summary | Network Redundancy

1 trips have 2 redundancies
2 trips have 1 redundancies
1 trips have 0 redundancies
1 trips have 3 redundancies



### Traffic Summary in the network

This is a summary of traffic on each **road edge** in the network. 

In [5]:
print(env.get_edge_traffic_summary())

Summary | Edge Traffic

Edge    Volume (%)    Travel Time
------------------------------
  0    40  (80.0%)      1.06      
  1    40  (80.0%)      1.06      
  2    30  (60.0%)      1.02      
  3    20  (40.0%)      1.00      
  4    40  (80.0%)      1.06      
  5    30  (60.0%)      1.02      
  6     0  (0.0%)      1.00      
  7    30  (60.0%)      1.02      
  8    30  (60.0%)      1.02      
  9    70  (140.0%)      1.58      
 10     0  (0.0%)      1.00      
 11    40  (80.0%)      1.06      



## Environment Step

In [6]:
from pprint import pprint

In [7]:
# reset the environment
env.reset()

# There are 5 actions for each component: 
# 0: do-nothing
# 1: inspect
# 2: minor-repair
# 3: major-repair 
# 4: replace

# Let us pick the minor-repair (2) action for all road segments
system_actions = []
for edge in env.graph.es:
    road_edge = edge["road_edge"]
    segment_actions = []
    for segment in road_edge.segments:
        segment_actions.append(2)

    system_actions.append(segment_actions)

print(f"System actions: {system_actions}")

# step in the environment
observation, reward, done, info = env.step(system_actions)

print("Observation:")
pprint(observation)
pprint(f"Reward: {reward}")
pprint(f"Done: {done}")
print("Info:")
pprint(info)

System actions: [[2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]
Observation:
{'adjacency_matrix': array([[0, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 0]]),
 'edge_beliefs': [[array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.7809482 , 0.19490781, 0.02414399, 0.        , 0.        ])],
                  [array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.97962555, 0.02037445, 0.        , 0.        , 0.        ])],
                  [array([0.7809482 , 0.19490781, 0.02414399, 0.        , 0.        ])],
             

## Environment Rollout

In [8]:
obs = env.reset()
done = False
timestep = 0

while not done:

    # system_actions = policy(observation)

    observation, reward, done, info = env.step(system_actions)

    timestep += 1

print(f"Number of timesteps: {timestep}")
print(f"Done: {done}")

Number of timesteps: 50
Done: True
